In [70]:
import pandas as pd
from openbb import obb
import plotly.graph_objects as go
import datetime

import numpy as np
from scipy.interpolate import griddata

In [71]:
def smooth_surface_rolling(surface, window_dte=3, window_strike=3, fill_median=True):
    """
    Apply a rolling mean smoothing on both DTE and strike axes.
    """
    surf = surface.copy()
    surf = surf.rolling(window=window_dte, axis=0, min_periods=1, center=True).mean()
    surf = surf.rolling(window=window_strike, axis=1, min_periods=1, center=True).mean()
    if fill_median:
        surf = surf.fillna(surface.stack().median())
    return surf

In [72]:
def interpolate_surface(surface, method='linear'):
    """
    Interpolate missing values on the volatility surface using griddata.
    """
    strikes = surface.columns.values
    dtes = surface.index.values
    X, Y = np.meshgrid(strikes, dtes)
    mask = ~surface.isna().values
    points = np.column_stack([X[mask], Y[mask]])
    values = surface.values[mask]
    target = np.column_stack([X.ravel(), Y.ravel()])
    interp_vals = griddata(points, values, target, method=method)
    surf_interp = pd.DataFrame(interp_vals.reshape(surface.shape), index=dtes, columns=strikes)
    # Fill any remaining NaNs with nearest neighbor
    missing = surf_interp.isna().values
    if missing.any():
        nearest = griddata(points, values, target, method='nearest')
        surf_interp.values[missing] = nearest.reshape(surface.shape)[missing]
    return surf_interp

In [2]:
obb.equity.search("Apple", provider="nasdaq").to_df()

,symbol,name,nasdaq_traded,exchange,market_category,etf,round_lot_size,test_issue,financial_status,nasdaq_symbol,next_shares,cqs_symbol
0,AAPL,Apple Inc. - Common Stock,Y,Q,Q,N,100.0,N,N,AAPL,N,NaN
1,AAPX,ETF Opportunities Trust T-Rex 2X Long Apple Da...,Y,Z,NaN,Y,100.0,N,NaN,AAPX,N,AAPX
2,AAPY,Kurv Yield Premium Strategy Apple (AAPL) ETF,Y,Z,NaN,Y,100.0,N,NaN,AAPY,N,AAPY
3,APLE,"Apple Hospitality REIT, Inc. Common Shares",Y,N,NaN,N,100.0,N,NaN,APLE,N,APLE
4,MLP,"Maui Land & Pineapple Company, Inc. Common Stock",Y,N,NaN,N,100.0,N,NaN,MLP,N,MLP
5,PAPL,Pineapple Financial Inc. Common Stock,Y,A,NaN,N,100.0,N,NaN,PAPL,N,PAPL


In [3]:
df = pd.DataFrame()

df["yfinance"] = (
    obb.equity.fundamental.balance(
        "TGT", provider="yfinance"
    )  # There is no limit for yFinance, historical data is limited.
    .to_df()
    .get("total_assets")
    .head(3)
)

df["fmp"] = (
    obb.equity.fundamental.balance("TGT", provider="fmp", limit=3)
    .to_df()
    .get("total_assets")
)

# too expensive (wichser)
# df["intrinio"] = (
#     obb.equity.fundamental.balance("TGT", provider="intrinio", limit=3)
#     .to_df()
#     .get("total_assets")
# )

df["polygon"] = (
    obb.equity.fundamental.balance("TGT", provider="polygon", limit=3)
    .to_df()
    .get("total_assets")
)

df

,yfinance,fmp,polygon
0,5.776900e+10,5.776900e+10,5.776900e+10
1,5.535600e+10,5.535600e+10,5.535600e+10
2,5.333500e+10,5.333500e+10,5.333500e+10


In [5]:
oc = obb.derivatives.options.chains("AAPL")

In [15]:
dd = oc.to_dict()

In [16]:
options = dd["CboeOptionsChainsData"]

In [17]:
len(options)

2228

In [18]:
options[0]

{'underlying_symbol': 'AAPL',
 'underlying_price': 210.0799,
 'contract_symbol': 'AAPL250502C00100000',
 'expiration': datetime.date(2025, 5, 2),
 'dte': 4,
 'strike': 100.0,
 'option_type': 'call',
 'open_interest': 4,
 'volume': 0,
 'theoretical_price': 110.1381,
 'last_trade_price': 98.7,
 'last_trade_time': '2025-04-22 13:48:30',
 'tick': 'up',
 'bid': 109.55,
 'bid_size': 101,
 'ask': 110.7,
 'ask_size': 101,
 'open': 0.0,
 'high': 0.0,
 'low': 0.0,
 'prev_close': 108.424999237061,
 'change': 0.0,
 'change_percent': 0.0,
 'implied_volatility': 0.0,
 'delta': 0.9997,
 'gamma': 0.0,
 'theta': 0.0,
 'vega': 0.0002,
 'rho': 0.0164}

In [19]:
df = pd.DataFrame(options)

In [47]:
df_puts = df[df["option_type"] == "put"].copy()
df_calls =  df[df["option_type"] == "call"].copy()

In [48]:
df_calls = df_calls[
    (df_calls["strike"] >= df_calls["underlying_price"] * 0.5) &
    (df_calls["strike"] <= df_calls["underlying_price"] * 1.5)
].copy()

In [54]:
df_calls.to_csv("calls.csv")

In [64]:
df_calls_146 = df_calls[df_calls["strike"] == 146.0]


In [50]:
# Für Puts
volatility_surface_puts = df_puts.pivot_table(
    values='implied_volatility',
    index='dte',
    columns='strike',
    aggfunc='mean'
)

volatility_surface_calls = df_calls.pivot_table(
    values='implied_volatility',
    index='dte',
    columns='strike',
    aggfunc='mean'
)


In [75]:
volatility_surface_calls = interpolate_surface(volatility_surface_calls)

In [67]:
strikes_with_nan = volatility_surface_calls.columns[
    volatility_surface_calls.isna().any(axis=0)
].tolist()

In [69]:
print(strikes_with_nan)

[115.0, 146.0, 147.0, 148.0, 149.0, 152.5, 157.5, 162.5, 167.5, 172.5, 177.5, 182.5, 187.5, 192.5, 197.5, 202.5, 205.0, 207.5, 212.5, 215.0, 217.5, 222.5, 225.0, 227.5, 235.0, 245.0, 255.0, 265.0, 275.0, 285.0, 295.0, 305.0, 315.0]


In [76]:

# Achsenwerte
expiration_dates_str = volatility_surface_calls.index.to_list()
strike_prices = volatility_surface_calls.columns.to_list()
volatility_values = volatility_surface_calls.values
# *** Verbesserungen für Ausrichtung und Darstellung ***

# X-Achse: Strike Preise (wie in der ersten Grafik)
x_axis_values = strike_prices
x_axis_title = 'Strike Preis'

# Y-Achse: Verfallsdaten (wie in der ersten Grafik, von "vorne" nach "hinten")
y_axis_values = expiration_dates_str
y_axis_title = 'Verfallsdatum'

# Z-Achse: Implizite Volatilität (Höhe)
z_axis_values = volatility_values
z_axis_title = 'Implizite Volatilität'


# 3. Plotly Surface Plot erstellen (angepasst)
fig = go.Figure(data=[go.Surface(z=z_axis_values,
                                    x=x_axis_values,  # Strike Preise auf X-Achse
                                    y=y_axis_values,  # Verfallsdaten auf Y-Achse
                                    colorscale='Viridis')])

# 4. Layout anpassen (verbessert für Ähnlichkeit zur ersten Grafik)
fig.update_layout(
    title='Volatilitätsoberfläche',
    scene=dict(
        xaxis_title=x_axis_title,
        yaxis_title=y_axis_title,
        zaxis_title=z_axis_title,
        xaxis=dict(tickformat=".0f"), # Keine Dezimalstellen bei Strike-Preisen
        yaxis=dict(
            tickvals=y_axis_values, # Explizite Tick-Werte für Verfallsdaten
            autorange='reversed' # Verfallsdaten von vorne nach hinten (früheste zuerst)
        ),
        camera=dict(                                 # Kameraperspektive anpassen
            eye=dict(x=1.2, y=-1.2, z=0.8)         # Position der Kamera (x, y, z Koordinaten)
        )
    ),
    margin=dict(l=20, r=20, b=20, t=40)
)

fig.show()